In [ ]:
!pip install earthengine-api

In [4]:
import ee
ee.Authenticate()  # Run this once, it will prompt you to authenticate via browser.
ee.Initialize()  # Initialize the API


Successfully saved authorization token.


In [11]:
import ee
import requests
import os
from datetime import datetime, timedelta 
import matplotlib.pyplot as plt

# Function do download satellite image from Google Earth Engine
def get_satellite_image(lat, lon, coordinate_offset, date, date_tolerance, save_dir='data/train_images', scale=500):

    # Make sure target directory exists
    os.makedirs(save_dir, exist_ok=True)

    # Create a rectangle around the coordinates to define the region of interest for the API call
    lat_min = lat - coordinate_offset
    lat_max = lat + coordinate_offset
    lon_min = lon - coordinate_offset
    lon_max = lon + coordinate_offset


    # Create a range of dates around the target date to allow for some flexibility

    date = datetime.strptime(date, "%Y-%m-%d") # convert string to date
    date_tolerance_delta = timedelta(days=date_tolerance)
    start_date = date - date_tolerance_delta
    end_date = date + date_tolerance_delta

    start_date = start_date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")

    # Create Geometry object
    region = ee.Geometry.Rectangle([lon_min, lat_min, lon_max, lat_max])
    
    # Obtain list of Sentinel-2 images by date and region
    image_collection = (
        ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")  # Updated Sentinel-2 dataset
        .filterBounds(region)
        .filterDate(start_date, end_date)
        .sort("CLOUDY_PIXEL_PERCENTAGE")  # Get least cloudy image
    )
    # check if there are images in the collection
    if image_collection.size().getInfo() == 0:
        print(f'No images found for the given date range {start_date} to {end_date} for coordinates {lat}, {lon}')
        return None
    else:
        #print(f"Found {image_collection.size().getInfo()} images")
        image = image_collection.first()
        #image = image = image_collection.mosaic()
        #image = image_collection.median()

    
    # Define visualization parameters (RGB bands, reduced scale)
    visualization = {
        "bands": ["B4", "B3", "B2"],  # Red, Green, Blue
        "min": 0,
        "max": 3000,
        "scale": scale  # Reduce resolution for smaller file size
    }
    
    # Get the download URL
    url = image.getThumbURL(visualization)
    
    # Define file name based on coordinates and date
    filename = f"sentinel_{lat}_{lon}_{date}_{date_tolerance}_{500}.png"
    filepath = os.path.join(save_dir, filename)

    # Download the image
    response = requests.get(url, stream=True)
    
    if response.status_code == 200:
        with open(filepath, "wb") as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)
        print(f"Image saved to {filepath}")
        return filepath
        # # show the image using matplotlib
        # img = plt.imread(filepath)
        # # Display the image in large figure
        # plt.figure(figsize=(10, 10))
        # return None
    else:
        print(f"Failed to download image. HTTP status code: {response.status_code}")
        return None

In [13]:
# Test the function
lat = 48.135124
lon = 11.581981
coordinate_offset = 0.0002 #size of the square area 
date = "2023-06-30"
date_tolerance = 30
save_dir = 'data/train_images'
scale = 500

get_satellite_image(lat, lon, coordinate_offset, date, date_tolerance, save_dir, scale=scale)

Image saved to data/train_images/sentinel_48.135124_11.581981_2023-06-30 00:00:00_30_500.png


<Figure size 1000x1000 with 0 Axes>

In [15]:
# Read Preprocessed Data with Wealth Index
import pandas as pd
senegal_cluster_wealth_data = pd.read_csv('data/Senegal_Wealth_Index.csv')
senegal_cluster_wealth_data



,DHSID,DHSCC,DHSYEAR,DHSCLUST,CCFIPS,ADM1FIPS,ADM1FIPSNA,ADM1SALBNA,ADM1SALBCO,ADM1DHS,...,URBAN_RURA,LATNUM,LONGNUM,ALT_GPS,ALT_DEM,DATUM,hv001,wealth_index,buffer,hv005
0,SN202300000001,SN,2023.0,1.0,SG,NaN,NaN,NaN,NaN,1.0,...,U,14.710267,-17.439639,9999.0,19.0,WGS84,1.0,1.945448,POLYGON ((-17.422413537528083 14.7035333721664...,1308253.0
1,SN202300000002,SN,2023.0,2.0,SG,NaN,NaN,NaN,NaN,1.0,...,U,14.715173,-17.462959,9999.0,21.0,WGS84,2.0,2.500621,POLYGON ((-17.444714764891764 14.7118413988093...,1308253.0
2,SN202300000003,SN,2023.0,3.0,SG,NaN,NaN,NaN,NaN,1.0,...,U,14.699729,-17.443807,9999.0,9.0,WGS84,3.0,2.878297,POLYGON ((-17.42556342080701 14.69639594998376...,1308253.0
3,SN202300000004,SN,2023.0,4.0,SG,NaN,NaN,NaN,NaN,1.0,...,U,14.724768,-17.447563,9999.0,18.0,WGS84,4.0,2.018379,POLYGON ((-17.429317991276733 14.7214356427084...,1308253.0
4,SN202300000005,SN,2023.0,5.0,SG,NaN,NaN,NaN,NaN,1.0,...,U,14.698593,-17.461012,9999.0,21.0,WGS84,5.0,1.784723,POLYGON ((-17.442768791733176 14.6952613946240...,1308253.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083,SN202300000393,SN,2023.0,393.0,SG,NaN,NaN,NaN,NaN,14.0,...,R,12.759350,-15.369216,9999.0,32.0,WGS84,393.0,-1.408346,POLYGON ((-15.427542875290415 12.6893610120072...,1971429.0
1084,SN202300000393,SN,2023.0,393.0,SG,NaN,NaN,NaN,NaN,14.0,...,R,12.759350,-15.369216,9999.0,32.0,WGS84,393.0,-1.109570,POLYGON ((-15.45439067430258 12.79382164157992...,1971429.0
1085,SN202300000394,SN,2023.0,394.0,SG,NaN,NaN,NaN,NaN,14.0,...,R,12.721166,-15.508203,9999.0,21.0,WGS84,394.0,-1.589147,POLYGON ((-15.449645525449641 12.6513705709041...,1971429.0
1086,SN202300000394,SN,2023.0,394.0,SG,NaN,NaN,NaN,NaN,14.0,...,R,12.721166,-15.508203,9999.0,21.0,WGS84,394.0,-2.159073,POLYGON ((-15.600312299678174 12.7209729828417...,1971429.0


In [16]:
coordinate_offset = 0.0002
date = "2023-06-30"
date_tolerance = 30
save_dir = 'data/train_images'
scale = 500

filepath_list = []

# Loop through each row in the dataframe and download the satellite image
for idx, row in senegal_cluster_wealth_data.iterrows():
    lat = row['LATNUM']
    lon = row['LONGNUM']
    save_location = get_satellite_image(lat, lon, coordinate_offset, date, date_tolerance, save_dir, scale=scale)
    filepath_list.append(save_location)
    print(f"Image {idx+1} of {len(senegal_cluster_wealth_data)}")

# Add filepath list to the dataframe
senegal_cluster_wealth_data['filepath'] = filepath_list


Image saved to data/train_images/sentinel_14.710267236_-17.439638665_2023-06-30 00:00:00_30_500.png
Image 1 of 1088
Image saved to data/train_images/sentinel_14.715173084_-17.462959238_2023-06-30 00:00:00_30_500.png
Image 2 of 1088
Image saved to data/train_images/sentinel_14.699729391_-17.443806525_2023-06-30 00:00:00_30_500.png
Image 3 of 1088
Image saved to data/train_images/sentinel_14.724768453_-17.44756325_2023-06-30 00:00:00_30_500.png
Image 4 of 1088
Image saved to data/train_images/sentinel_14.698593455_-17.46101184_2023-06-30 00:00:00_30_500.png
Image 5 of 1088
Image saved to data/train_images/sentinel_14.737850237_-17.444763744_2023-06-30 00:00:00_30_500.png
Image 6 of 1088
Image saved to data/train_images/sentinel_14.735190373_-17.462996632_2023-06-30 00:00:00_30_500.png
Image 7 of 1088
Image saved to data/train_images/sentinel_14.740219725_-17.447833436_2023-06-30 00:00:00_30_500.png
Image 8 of 1088
Image saved to data/train_images/sentinel_14.734646961_-17.415766538_2023-

KeyboardInterrupt: 

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

...

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>